In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "3"
import tensorflow as tf
print(tf.__version__)
import keras
print(keras.__version__)

/home/sunxin/miniconda3/envs/qinhanmin-test/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/sunxin/miniconda3/envs/qinhanmin-test/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/sunxin/miniconda3/envs/qinhanmin-test/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/sunxin/minico

1.13.2
2.2.4


Using TensorFlow backend.


In [4]:
import keras
from keras.datasets import mnist
from keras import models
from keras import layers
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV

In [5]:
num_classes = 10

# input image dimensions
img_rows, img_cols = 28, 28

# load training data and do basic data normalization
(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [6]:
def make_model(dense_layer_sizes, filters, kernel_size, pool_size):
    '''Creates model comprised of 2 convolutional layers followed by dense layers
    dense_layer_sizes: List of layer sizes.
        This list has one number for each layer
    filters: Number of convolutional filters in each convolutional layer
    kernel_size: Convolutional kernel size
    pool_size: Size of pooling area for max pooling
    '''

    model = models.Sequential()
    model.add(layers.Conv2D(filters, kernel_size,
                            padding='valid',
                            input_shape=input_shape))
    model.add(layers.Activation('relu'))
    model.add(layers.Conv2D(filters, kernel_size))
    model.add(layers.Activation('relu'))
    model.add(layers.MaxPooling2D(pool_size=pool_size))
    model.add(layers.Dropout(0.25))

    model.add(layers.Flatten())
    for layer_size in dense_layer_sizes:
        model.add(layers.Dense(layer_size))
        model.add(layers.Activation('relu'))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(num_classes))
    model.add(layers.Activation('softmax'))

    model.compile(loss='categorical_crossentropy',
                  optimizer='adadelta',
                  metrics=['accuracy'])
    return model

In [7]:
dense_size_candidates = [[32], [64], [32, 32], [64, 64]]
my_classifier = KerasClassifier(make_model, batch_size=32)
validator = GridSearchCV(my_classifier,
                         param_grid={'dense_layer_sizes': dense_size_candidates,
                                     # epochs is avail for tuning even when not
                                     # an argument to model building function
                                     'epochs': [3, 6],
                                     'filters': [8],
                                     'kernel_size': [3],
                                     'pool_size': [2]},
                         scoring='neg_log_loss',
                         n_jobs=1)
validator.fit(x_train, y_train)

print('The parameters of the best model are: ')
print(validator.best_params_)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


/home/sunxin/miniconda3/envs/qinhanmin-test/lib/python3.7/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


Instructions for updating:
Use tf.cast instead.
Epoch 1/3
40000/40000 [==============================] - 9s 237us/step - loss: 0.6343 - acc: 0.7911
Epoch 2/3
40000/40000 [==============================] - 7s 183us/step - loss: 0.3780 - acc: 0.8793
Epoch 3/3
40000/40000 [==============================] - 7s 182us/step - loss: 0.3114 - acc: 0.9017
Epoch 1/3
40000/40000 [==============================] - 8s 191us/step - loss: 0.7124 - acc: 0.7607
Epoch 2/3
40000/40000 [==============================] - 7s 184us/step - loss: 0.3921 - acc: 0.8736
Epoch 3/3
40000/40000 [==============================] - 7s 187us/step - loss: 0.3130 - acc: 0.9034
Epoch 1/3
40000/40000 [==============================] - 8s 195us/step - loss: 0.6531 - acc: 0.7866
Epoch 2/3
40000/40000 [==============================] - 7s 185us/step - loss: 0.3753 - acc: 0.8829
Epoch 3/3
40000/40000 [==============================] - 7s 186us/step - loss: 0.3016 - acc: 0.9097
Epoch 1/6
40000/40000 [=============================

40000/40000 [==============================] - 9s 214us/step - loss: 0.1630 - acc: 0.9499
Epoch 6/6
40000/40000 [==============================] - 8s 212us/step - loss: 0.1497 - acc: 0.9555
Epoch 1/3
40000/40000 [==============================] - 10s 249us/step - loss: 0.4054 - acc: 0.8763
Epoch 2/3
40000/40000 [==============================] - 9s 217us/step - loss: 0.1727 - acc: 0.9527
Epoch 3/3
40000/40000 [==============================] - 9s 215us/step - loss: 0.1291 - acc: 0.9655
Epoch 1/3
40000/40000 [==============================] - 10s 249us/step - loss: 0.4359 - acc: 0.8665
Epoch 2/3
40000/40000 [==============================] - 9s 217us/step - loss: 0.1746 - acc: 0.9511
Epoch 3/3
40000/40000 [==============================] - 9s 214us/step - loss: 0.1329 - acc: 0.9638
Epoch 1/3
40000/40000 [==============================] - 10s 254us/step - loss: 0.3810 - acc: 0.8852
Epoch 2/3
40000/40000 [==============================] - 9s 218us/step - loss: 0.1539 - acc: 0.9578
Epoch 3

In [8]:
# validator.best_estimator_ returns sklearn-wrapped version of best model.
# validator.best_estimator_.model returns the (unwrapped) keras model
best_model = validator.best_estimator_.model
metric_names = best_model.metrics_names
metric_values = best_model.evaluate(x_test, y_test)
for metric, value in zip(metric_names, metric_values):
    print(metric, ': ', value)

10000/10000 [==============================] - 1s 140us/step
loss :  0.04639862526347424
acc :  0.986
